In [2]:
import tensorflow as tf
from tensorflow.python.platform import build_info as build

tf.autograph.set_verbosity(0)

print(f"tensorflow version: {tf.__version__}")
print(f"Cuda Version: {build.build_info['cuda_version']}")
print(f"Cudnn version: {build.build_info['cudnn_version']}")

tensorflow version: 2.15.0
Cuda Version: 12.2
Cudnn version: 8


In [ ]:
import tensorflow as tf
import tf2onnx
import onnx

In [ ]:
model = tf.keras.models.load_model('test.keras')

In [ ]:
if isinstance(model.input, list):
    input_details = [{'shape': input_tensor.shape, 'dtype': input_tensor.dtype, 'name': input_tensor.name} for input_tensor in model.inputs]
else:
    input_details = [{'shape': model.input.shape, 'dtype': model.input.dtype, 'name': model.input.name}]

# Print input details
for i, detail in enumerate(input_details):
    print(f"Input {i+1}: shape: {detail['shape']}, dtype: {detail['dtype']}, name: {detail['name']}")


In [ ]:
input_signature = [
    tf.TensorSpec(shape=(None, 807), dtype=tf.float32, name='review'),
    tf.TensorSpec(shape=(None, 1), dtype=tf.float32, name='types')
]

In [ ]:
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=18)
onnx.save(onnx_model, 'test.onnx')

In [ ]:
# !python -m tf2onnx.convert --saved-model saved-model --output model.onnx

In [ ]:
!pip show onnxruntime-gpu

In [ ]:
import onnxruntime as ort

session = ort.InferenceSession('test.onnx', providers=['CUDAExecutionProvider'])

In [ ]:
import numpy as np

# Dummy input data (replace with real data)
review_input = np.random.rand(1, 807).astype(np.float32)
types_input = np.random.rand(1, 1).astype(np.float32)

# Prepare inputs
inputs = {
    "review": review_input,
    "types": types_input
}

results_ort = session.run(None, inputs)

print(results_ort)

In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# from keras_tuner import RandomSearch, HyperModel

# Load the dataset
file_path = 'combined-dataset/final_reviews_data.csv'
data = pd.read_csv(file_path)

# Encode the 'types' column
label_encoder = LabelEncoder()
data['types_encoded'] = label_encoder.fit_transform(data['types'])

# Tokenize the 'review' column
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(data['review'])

# Pad the sequences
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Create the feature set
X = {
    'review': padded_sequences.astype(np.float32),
    'types': data['types_encoded'].values.astype(np.float32),
}

# Normalize the sentiment scores
y = data['sentiment'].values


In [ ]:
df_review = pd.read_csv('combined-dataset/final_reviews_data.csv')
df_place = pd.read_csv('combined-dataset/combined_datasetV2.csv')

In [ ]:
import numpy as np

# Dummy input data (replace with real data)
review_input = X['review']
types_input = X['types'].reshape(-1, 1)

# Prepare inputs
inputs = {
    "review": review_input,
    "types": types_input
}

results_ort = session.run(None, inputs)

print(results_ort)